In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import gradio as gr

In [ ]:
import zipfile
import os

In [ ]:
import os
print(os.path.getsize("/content/model.zip"))

119537664


In [ ]:
!file /content/model.zip

/content/model.zip: Zip archive data, at least v1.0 to extract, compression method=store


In [ ]:
with zipfile.ZipFile("/content/model.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

BadZipFile: File is not a zip file

In [ ]:
model_path = "./political_orientation_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
inverse_label_mapping = {
    0: 'communist',
    1: 'conservative',
    2: 'liberal',
    3: 'other',
    4: 'right wing populist',
    5: 'social democtratic',
}

In [ ]:
def predict_political_orientation(speech_text):
    """
    Predict the political orientation of the given speech text.
    """
    try:
        model.eval()  # Set the model to evaluation mode

        # Tokenize and move inputs to the appropriate device
        inputs = tokenizer(
            speech_text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

            # Move logits to CPU before converting to NumPy
            probabilities = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()[0]
            predicted_class = torch.argmax(logits, dim=1).item()

        # Convert predicted class ID to the label name
        prediction_label = inverse_label_mapping.get(predicted_class, "Unknown")
        return f"Predicted Political Orientation: {prediction_label}"

    except Exception as e:
        # Return the error message for easier debugging
        return f"An error occurred during prediction: {str(e)}"

In [ ]:
interface = gr.Interface(
    fn=predict_political_orientation,  # Your prediction function
    inputs=gr.Textbox(lines=10, placeholder='Enter the speech here...'),
    outputs=gr.Textbox(label="Prediction"),
    title='Political Orientation Predictor',
    description='Enter a speech and predict its political orientation.',
    examples=[
        ['This government is working tirelessly for the people.'],
        ['The workers need to rise against the oppressive capitalists.'],
        ['We stand by the conservative values of family and nation.'],
    ]
)

In [ ]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63e95390166035adf9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
